In [ ]:
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np

# Replace with paths to your model and label map
model_path = "path/to/your/model.pb"
label_map_path = "path/to/your/label_map.txt"

# Load the model and labels
net = cv2.dnn.readNetFromTensorflow(model_path)
categories = []
with open(label_map_path, 'r') as f:
    for line in f:
        categories.append(line.strip())

app = FastAPI()

def preprocess_image(image_bytes):
    # Read image from bytes
    image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
    # Preprocess the image (resize) based on your model requirements
    image = cv2.resize(image, (300, 300))
    image_blob = cv2.dnn.blobFromImage(image, swapRB=True)  # BGR -> RGB for OpenCV
    return image_blob

def predict_objects(image):
    net.setInput(image)
    outputs = net.forward()
    boxes = []
    confidences = []
    class_ids = []
    for detection in outputs[0, 0, :, :]:
        confidence = detection[2]
        if confidence > 0.5:  # Adjust threshold as needed
            box = detection[3:7] * np.array([image.shape[0], image.shape[1], image.shape[0], image.shape[1]])
            class_id = int(detection[1])
            boxes.append(box.astype(int))
            confidences.append(confidence)
            class_ids.append(class_ids)

    results = []
    for i in range(len(boxes)):
        x_min, y_min, x_max, y_max = boxes[i]
        class_name = categories[class_ids[i]]
        confidence = confidences[i]
        results.append({"class": class_name, "score": confidence, "bbox": [x_min, y_min, x_max, y_max]})
    return results

@app.post("/upload")
async def upload_image(image: UploadFile = File(...)):
    # Read image content
    image_bytes = await image.read()
    # Preprocess image
    image_blob = preprocess_image(image_bytes)
    # Perform object detection
    results = predict_objects(image_blob)
    return results

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000)

